In [30]:
import os
import subprocess

import json
import csv

import uuid

from IPython.display import display_javascript, display_html, display

import pandas as pd
import numpy as np

from datetime import datetime, date, time

In [31]:
from platform import python_version
print(python_version())

3.10.11


In [32]:
! pip install git+https://github.com/JustAnotherArchivist/snscrape.git

  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git 'C:\Users\PATRICIA\AppData\Local\Temp\pip-req-build-_v702nu4'



  Cloning https://github.com/JustAnotherArchivist/snscrape.git to c:\users\patricia\appdata\local\temp\pip-req-build-_v702nu4
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit 614d4c2029a62d348ca56598f87c425966aaec66
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


In [33]:
import snscrape.modules.twitter as sntwitter

In [34]:
#Run the snscrape help to see what options / parameters we can use
cmd = 'snscrape --help'

#This is similar to running os.system(cmd), which would show the output of running the command in the Terminal
#window from where I started my Jupyter Notebook (which is what I used to develop this code)
#By using subprocees, I capture the commands's output into a variable, whose content I can then print here.
output = subprocess.check_output(cmd, shell=True)
                                 
print(output.decode("utf-8")) 

usage: snscrape [-h] [--version] [--citation] [-v] [--dump-locals] [--retry N]
                [-n N] [-f FORMAT | --jsonl | --jsonl-for-buggy-int-parser]
                [--with-entity] [--since DATETIME] [--progress]
                SCRAPER ...

options:
  -h, --help            show this help message and exit
  --version             show program's version number and exit
  --citation            Display recommended citation information and exit
                        (default: None)
  -v, --verbose, --verbosity
                        Increase output verbosity (default: 0)
  --dump-locals         Dump local variables on serious log messages (warnings
                        or higher) (default: False)
  --retry N, --retries N
                        When the connection fails or the server returns an
                        unexpected response, retry up to N times with an
                        exponential backoff (default: 3)
  -n N, --max-results N
                        Only retu

In [35]:
filename = 'data/raw/tweets/bitcoins_query-tweets.jsonl'
os.makedirs(os.path.dirname(filename), exist_ok=True)

#Using the OS library to call CLI commands in Python
os.system(f'snscrape --max-results 100 --jsonl --progress --since 2018-12-01 twitter-search "#bitcoins lang:fr until:2019-12-31" > {filename}')

1

In [36]:
start = date(2018, 1, 1)
start = start.strftime('%Y-%m-%d')

stop = date(2024, 12, 31)
stop = stop.strftime('%Y-%m-%d')

keyword = 'bitcoins'

In [39]:
maxTweets = 100

#We are going to write the data into a JSON file
filename = 'data/raw/tweets/bitcoins_query-tweets.jsonl'
os.makedirs(os.path.dirname(filename), exist_ok=True)

# Set up the query for snscrape
query = 'volatilité bitcoin lang:fr -is:retweet'
# Other parameters for filtering tweets
min_likes = 20  # Nombre minimum de likes pour filtrer les tweets
lang = "fr"  # Langue des tweets à récupérer
since_year = 2018  # Année de début pour les tweets
until_year = 2024  # Année de fin pour les tweets

search_query = f"{query} min_retweets:0 min_faves:{min_likes} lang:{lang} since:{since_year}-01-01 until:{until_year}-12-31"
#We write to the JSON file by using JSONL format, which is a convenient way to store JSON objects in a line-delimited format.
with open(filename, "w", encoding="utf-8") as f:
#I will use the following Twitter search operators:
# since - start date for Tweets collection 
# stop  - stop date for Tweets collection
# -filter:links - not very clear what this does, from Twitter search operators documentation: https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/search-operators
#                 but it looks like this will exclude tweets with links from the search results
# -filter:replies - removes @reply tweets from search results
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(search_query).get_items()):
        if i > 10 :
            break
        tweet_data = {
        "Id": tweet.id,
        "content": tweet.content,
        "Likes": tweet.likeCount,
        "Retweets": tweet.retweetCount,
        "Date": tweet.date.isoformat(),
        "Created_at": tweet.date.isoformat(),
        "raw_content": tweet.rawContent,
        "replies_count": tweet.replyCount,
        "retweeted_count": tweet.retweetCount,
        "likes_count": tweet.likeCount,
        "hashtags": tweet.hashtags,
        "cash_tags": tweet.cashtags if tweet.cashtags else [],  # Vérifier si les cashtags existent
        "Language": tweet.lang,
        "link": tweet.url,
        "conversation_id": tweet.conversationId,
        "jahr": tweet.date.year,
        "monat": tweet.date.month,
        "tag": tweet.date.day,
        "time": tweet.date.time().isoformat(),
        "user_id": tweet.user.id,
        "user_name": tweet.user.username,
    }  
        
        f.write(json.dumps(tweet_data, ensure_ascii=False) + "\n")  # Écrire le tweet dans le fichier JSON


Error retrieving https://twitter.com/search?f=live&lang=en&q=volatilit%C3%A9+bitcoin+lang%3Afr+-is%3Aretweet+min_retweets%3A0+min_faves%3A20+lang%3Afr+since%3A2018-01-01+until%3A2024-12-31&src=spelling_expansion_revert_click: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com', port=443): Max retries exceeded with url: /search?f=live&lang=en&q=volatilit%C3%A9+bitcoin+lang%3Afr+-is%3Aretweet+min_retweets%3A0+min_faves%3A20+lang%3Afr+since%3A2018-01-01+until%3A2024-12-31&src=spelling_expansion_revert_click (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))"))
4 requests to https://twitter.com/search?f=live&lang=en&q=volatilit%C3%A9+bitcoin+lang%3Afr+-is%3Aretweet+min_retweets%3A0+min_faves%3A20+lang%3Afr+since%3A2018-01-01+until%3A2024-12-31&src=spelling_expansion_revert_click failed, giving up.
Errors: SSLError(MaxRetryError("HTTPSConnectionPool(host='twitter.com

ScraperException: 4 requests to https://twitter.com/search?f=live&lang=en&q=volatilit%C3%A9+bitcoin+lang%3Afr+-is%3Aretweet+min_retweets%3A0+min_faves%3A20+lang%3Afr+since%3A2018-01-01+until%3A2024-12-31&src=spelling_expansion_revert_click failed, giving up.